In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
summertrip2021 = pd.read_csv(r'.\All_Routes_Trip_Summary_Summer2021.csv')
falltrip2021 = pd.read_csv(r'.\City_Trip_Summary_Fall2021.csv')

In [ ]:
list(summertrip2021.columns)

In [ ]:
summertrip2021 = summertrip2021.drop(columns=["Unnamed: 0"])

In [ ]:
summertrip2021.head()

In [ ]:
codes=falltrip2021["Trip Code"].unique()

In [ ]:
for i in codes:
    print(i)

In [ ]:
len(codes)

In [ ]:
code_glimpse = trip2021.groupby(by="Trip Code", as_index=False).count().sort_values(by="Route", ascending=False)

code_glimpse=code_glimpse[["Trip Code", "Route"]]
code_glimpse.head()

In [ ]:
code_glimpse["Route"].unique()

In [ ]:
bus_stops = pd.read_csv(r'.\data\bge_busstops.csv')

In [ ]:
bus_stops = bus_stops.drop(columns=["the_geom_2272", "the_geom"])

In [ ]:
xsect = bus_stops.loc[bus_stops["category"] == "intersection"]
xsect

In [ ]:
target = []

for name in xsect.stop_name:
    target.append(name)
    
print(target)

In [ ]:
targetlist = summertrip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in target))

In [ ]:
summerBGE = summertrip2021[targetlist]
summerBGE.to_csv('summerBGE.csv', index=False)

In [ ]:
summerBGE["Stop Seq."].unique()

# row binding the two new DFs fall and summer bus stops

In [ ]:
summerBGE = pd.read_csv(r'.\summerBGE.csv')
fallBGE = pd.read_csv(r'.\fallBGE.csv')

In [ ]:
allBGE = pd.concat([summerBGE, fallBGE])

In [ ]:
len(allBGE)

In [ ]:
summer = len(summerBGE)
fall = len(fallBGE)
summer + fall

### it worked! let's export

In [ ]:
allBGE.to_csv('allBGE.csv', index=False)

In [ ]:
allBGE.head()

 # getting rid of the weird numbers in the stop seq

In [ ]:
allBGE = allBGE.rename(columns = {'Stop Seq.': 'StopSeq'}) 

In [ ]:
import re
regex = re.compile('[^a-zA-Z]')

In [ ]:
stopseq = []
for stop in allBGE.StopSeq: 
    newstop = re.sub("[^a-zA-Z]+", "", stop)
    stopseq.append(newstop)


In [ ]:
allBGE["stop_name"] = stopseq

In [ ]:
allBGE.head()

## this got rid of the & and the -, so it will be hard to match with the bustops table, so I will just get rid of those symbols int he bus stop table

In [ ]:
stopseq2 = []
for stop in xsect.stop_name: 
    newstop = re.sub("[^a-zA-Z]+", "", stop)
    stopseq2.append(newstop)


In [ ]:
stopseq2

In [ ]:
xsect["cleanstop_name"] = stopseq2

In [ ]:
xsect.cleanstop_name.unique

## now merging them together to get those geoms 

In [171]:
allBGE.geom = allBGE.join( xsect, rsuffix= "cleanstop_name", lsuffix = "stop_name")


In [172]:
print(allBGE.geom.columns)

Index(['Route', 'Direction', 'Pattern', 'Trip', 'Trip Code', 'Trip Type',
       'Stop Code', 'StopSeq', 'Avg. Ons', 'Avg. Offs', 'Load', 'Dwell Time',
       'Sch. Start Time', 'Sch. End Time', 'Act. Start Time', 'Act. End Time',
       'Time Dev', 'Sch. Run Time', 'Act. Run Time', 'Run Dev',
       'stop_namestop_name', 'stop_id', 'stop_namecleanstop_name', 'stop_lat',
       'stop_lon', 'location_type', 'parent_station', 'zone_id',
       'wheelchair_boarding', 'dist', 'category', 'cleanstop_name'],
      dtype='object')


In [173]:
allBGE.geom = allBGE.geom.drop(columns= ['location_type', 'parent_station', 'zone_id', 'wheelchair_boarding', 'dist'])

In [174]:
len(allBGE.geom)

4238

In [175]:
len(allBGE)

4238

### it looks like something weird happened but i might not be a big deal. it looks like the merge to get the geom added a 1000 rows?

In [176]:
#### exporting anyways
allBGE.geom.to_csv('allBGE_geom.csv', index=False)

# Getting the Center & North Bus stops the same way

In [ ]:
central_stops = pd.read_csv(r'.\data\central_stops.csv')

In [ ]:
allnorth_stops = pd.read_csv(r'.\data\allnorth_stops.csv')


In [122]:
targetcentral = []
for name in central_stops.stop_name:
    targetcentral.append(name)
    
print(targetcentral)

['17th St & Spring Garden St', '17th St & Market St', '17th St & Pine St', '17th St & Washington Av', '16th St & Pine St ', '2nd St & Green St', 'Front St & Market St Loop', '3rd St & Spring Garden St', '23rd St & Market St - FS ', '23rd St & Washington Av', 'Arch St & 22nd St', 'Walnut St & Broad St - MBNS', 'Walnut St & 23rd St', 'Spruce St & 15th St', 'Broad St & Washington Av ', '13th St Trolley Station', 'Locust St & 20th St', 'Locust St & Broad St ', 'Pine St & 3rd St', '23rd St & Spruce St', '20th St & Walnut St', 'Market St & 6th St', '19th St & Washington Av', '2nd St Station - MFL', '15th St & Market St', '27th St & Poplar St', "Penn's Landing - 1", '5th St & Market St - FS', 'Art Museum Dr & Back Entrance - 2', 'South St & 22nd St', 'South St & Broad St', 'Lombard St & Broad St', 'Lombard St & 22nd St', 'Spruce St & 2nd St', 'Spring Garden St & 22nd St - FS', 'Spring Garden St & Broad St', 'Beach St & Columbia Av', 'Spring Garden St & Front St', 'Spring Garden St & Broad St'

In [121]:
targetallnorth = []
for name in allnorth_stops.stop_name:
    targetallnorth.append(name)
    
print(targetallnorth)

['Roosevelt Blvd & 5th St', 'Ogontz Av & 69th Av', 'Ogontz Av & Haines St', 'Ogontz Av & 66th Av', 'Ogontz Av & Stenton Av', 'Olney Transportation Center - BSL', 'Ogontz Av & Chelten Av - FS', 'Ogontz Av & 66th Av', 'Ogontz Av & Haines St', 'Fisher Av & Rising Sun Av - FS', 'Tabor Rd & 5th St', 'Tabor Rd & 5th St ', 'Fisher Av & Rising Sun Av', 'C St & Roosevelt Blvd', 'Roosevelt Blvd & C St', 'Roosevelt Blvd & Mascher St', 'Roosevelt Blvd & 2nd St', 'Roosevelt Blvd & 6th St', 'Roosevelt Blvd & 7th St - FS', 'Roosevelt Blvd & 9th St', 'Lindley Av & 5th St', 'Lindley Av & Broad St', 'Lindley Av & 16th St', 'Lindley Av & Broad St ', 'Lindley Av & 5th St', '66th Av & Broad St ', '66th Av & Old York Rd', 'Wister St & Chelten Av', 'Rodney St & Mt Airy Av', 'Roosevelt Blvd & 5th St - FS', 'Michener Av & Wadsworth Av - FS', 'Upsal St & Mansfield Av', 'Olney Av & Broad St', 'Olney Av & Broad St - FS east', 'Rising Sun Av & Olney Av', 'Olney Av & 13th St', 'Olney Transportation Center - onsit',

#### using the target lists

In [124]:
centraltargetlist = summertrip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in targetcentral))
summercentral = summertrip2021[centraltargetlist]
print("central summer done")

In [128]:
centraltargetlist = falltrip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in targetcentral))
fallcentral=  falltrip2021[centraltargetlist]
print("central fall done")

central fall done


In [129]:
allnorthtargetlist = summertrip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in targetallnorth))
summerallnorth =  summertrip2021[allnorthtargetlist]
print("allnorth summer done")

allnorth summer done


In [130]:
allnorthtargetlist = falltrip2021["Stop Seq."].apply(lambda sentence: any(word in sentence for word in targetallnorth))
fallallnorth = falltrip2021[allnorthtargetlist]
print("allnorth fall done")

allnorth fall done


In [126]:
summercentral["Stop Seq."].unique()

array(['(015) - Grays Ferry Av & Carpenter St ',
       '(016) - Grays Ferry Av & Christian St ',
       '(017) - Grays Ferry Av & Catherine St ', ...,
       '(047) - 13th St & Market St - MBFS',
       '(086) - 69th St & Walnut St - FS',
       '(088) - 69th St & Chestnut St '], dtype=object)

### now merging the summer and fall

In [131]:
allcentral = pd.concat([summercentral, fallcentral])
allallnorth = pd.concat([summerallnorth, fallallnorth])
print("done")

done


In [133]:
len(allcentral)

352401

In [132]:
summerc = len(summercentral)
fallc = len(fallcentral)
summerc + fallc

352401

In [134]:
len(allallnorth)

571069

In [135]:
summern = len(summerallnorth)
falln = len(fallallnorth)
summern + falln

571069

### it worked! let's export

In [136]:
allcentral.to_csv('allcentral_stops.csv', index=False)
allallnorth.to_csv('allallnorth_stops.csv', index=False)

 # Round 2: getting rid of the weird numbers in the stop seq

In [137]:
allcentral = allcentral.rename(columns = {'Stop Seq.': 'StopSeq'}) 
allallnorth = allallnorth.rename(columns = {'Stop Seq.': 'StopSeq'}) 
print("done")

done


In [138]:
import re
regex = re.compile('[^a-zA-Z]')

In [139]:
stopseqC = []
for stop in allcentral.StopSeq: 
    newstop = re.sub("[^a-zA-Z]+", "", stop)
    stopseqC.append(newstop)
print("done")

done


In [140]:
stopseqN = []
for stop in allallnorth.StopSeq: 
    newstop = re.sub("[^a-zA-Z]+", "", stop)
    stopseqN.append(newstop)
print("done")

done


In [141]:
allcentral["stop_name"] = stopseqC
allallnorth["stop_name"] = stopseqN
print("done")

done


## this got rid of the & and the -, so it will be hard to match with the bustops table, so I will just get rid of those symbols int he bus stop table

In [142]:
allseptabus = pd.read_csv(r'.\data\septa_bus_stops_all.csv')


In [143]:
stopseq3 = []
for stop in allseptabus.stop_name: 
    newstop = re.sub("[^a-zA-Z]+", "", stop)
    stopseq3.append(newstop)
print('done')


done


In [144]:
allseptabus["cleanstop_name"] = stopseq3

## now merging them together to get those geoms 

In [162]:
allcentral.geom = allcentral.join( allseptabus, rsuffix= "cleanstop_name", lsuffix = "stop_name")
allallnorth.geom = allallnorth.join( allseptabus, rsuffix= "cleanstop_name", lsuffix = "stop_name")
print("done")

done


In [177]:
allallnorth.geom.head()

,Unnamed: 0,Route,Direction,Pattern,Trip,Trip Code,Trip Type,Stop Code,StopSeq,Avg. Ons,...,stop_namecleanstop_name,stop_lat,stop_lon,location_type,parent_station,zone_id,wheelchair_boarding,the_geom,the_geom_2272,cleanstop_name
0,1.0,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,4st,(001) - Roosevelt Blvd & Broad St - FS,9.696970,...,Roosevelt Blvd & Comly Rd,40.105172,-75.001575,NaN,NaN,1.0,1.0,0101000020E6100000705F07CE19C052C03B3AAE46760D...,0101000020E0080000705F07CE19C052C03B3AAE46760D...,RooseveltBlvdComlyRd
0,NaN,1,Northbound,N08,[05:13] 8670224,8670224,Revenue,4st,(001) - Roosevelt Blvd & Broad St - FS,9.000000,...,Roosevelt Blvd & Comly Rd,40.105172,-75.001575,NaN,NaN,1.0,1.0,0101000020E6100000705F07CE19C052C03B3AAE46760D...,0101000020E0080000705F07CE19C052C03B3AAE46760D...,RooseveltBlvdComlyRd
1,2.0,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,22406st,(002) - Roosevelt Blvd & 9th St,2.303030,...,Roosevelt Blvd & Solly Av - FS,40.058873,-75.045067,NaN,NaN,1.0,1.0,0101000020E61000003FC8B260E2C252C007CF84268907...,0101000020E00800003FC8B260E2C252C007CF84268907...,RooseveltBlvdSollyAvFS
1,NaN,1,Northbound,N08,[05:13] 8670224,8670224,Revenue,22406st,(002) - Roosevelt Blvd & 9th St,2.235294,...,Roosevelt Blvd & Solly Av - FS,40.058873,-75.045067,NaN,NaN,1.0,1.0,0101000020E61000003FC8B260E2C252C007CF84268907...,0101000020E00800003FC8B260E2C252C007CF84268907...,RooseveltBlvdSollyAvFS
2,3.0,1,Northbound,N08,[05:13] 8548782,8548782,Revenue,23308st,(003) - Roosevelt Blvd & Wyoming Av,0.000000,...,Roosevelt Blvd & Cottman Av,40.044862,-75.054530,NaN,NaN,1.0,1.0,0101000020E6100000A8A9656B7DC352C0A56ABB09BE05...,0101000020E0080000A8A9656B7DC352C0A56ABB09BE05...,RooseveltBlvdCottmanAv


In [178]:
allcentral.geom = allcentral.geom.drop(columns= ['Unnamed: 0', 'location_type', 'parent_station', 'zone_id', 'wheelchair_boarding','the_geom_2272'])
allallnorth.geom = allallnorth.geom.drop(columns= ['Unnamed: 0','location_type', 'parent_station', 'zone_id', 'wheelchair_boarding','the_geom_2272'])

In [181]:
len(allcentral)

352401

In [182]:
len(allcentral.geom)

352401

In [183]:
allcentral.geom.to_csv('allcentral_stops_geom.csv', index=False)
allallnorth.geom.to_csv('allallnorth_stops_geom.csv', index=False)

# everything below is not clean

In [ ]:
print(allBGE.geom.columns)

In [ ]:
allBGE.geom = allBGE.geom.drop(columns= ['location_type', 'parent_station', 'zone_id', 'wheelchair_boarding', 'dist', 'stop_name_y'])

In [154]:
len(allcentral.geom)

4210665

In [155]:
len(allcentral)

352401

In [ ]:
allBGE.geom.isnull().values.any()

In [ ]:
allBGE["stop_name"] = stopseq

In [ ]:
allBGE.head()

# This is just extra stuff that helped me figure stuff out

In [ ]:
 Erie = trip2021["Stop Seq."].str.contains("Erie", case=True)

In [ ]:
German = trip2021["Stop Seq."].str.contains("German", case=True)

In [ ]:
GermanErie= trip2021["Stop Seq."].str.contains("German|Erie", case=True)

In [ ]:
GE = ['Germantown', 'Erie']

In [ ]:
GElist = trip2021["Stop Seq."].apply(lambda sentence: all(word in sentence for word in GE))

In [ ]:
XHErie = ['XH', 'Erie']

In [ ]:
XHErielist = trip2021["Stop Seq."].apply(lambda sentence: all(word in sentence for word in XHErie))